In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
loader = TextLoader("./data/dswithbappy_data.txt")
loaded_document = loader.load()

In [4]:
loaded_document

[Document(metadata={'source': './data/dswithbappy_data.txt'}, page_content='dswithbappy focuses on providing content on Data Science, Artificial Intelligence, Machine Learning, \nDeep Learning, Computer Vision, Natural language processing, Python programming, etc. in English. \n"Real creativity won\'t make things more complex. Instead, I will simplify them." \nI have been working in the Data Science field, spearheading in machine learning, \ndeep learning, and computer vision. Typically this is my YouTube channel where I clarify \ndifferent themes on machine learning, deep learning, and AI with numerous real-world issue scenarios. \nMy primary point is to create everybody commonplace of ML and AI. Please subscribe and support the channel. \nAs I come up with new innovations. These recordings are free and I guarantee to create more curiously substance as we go ahead.\n😀Please donate if you want to support the channel through Buy me a coffee: https://www.buymeacoffee.com/dswithbappy\n\nP

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [6]:
len(chunks_of_text)

1

In [7]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [8]:
vector_db

In [9]:
retriever = vector_db.as_retriever()

In [10]:
response = retriever.invoke("what is dswithbappy?")

In [11]:
response

[Document(metadata={'source': './data/dswithbappy_data.txt'}, page_content='dswithbappy focuses on providing content on Data Science, Artificial Intelligence, Machine Learning, \nDeep Learning, Computer Vision, Natural language processing, Python programming, etc. in English. \n"Real creativity won\'t make things more complex. Instead, I will simplify them." \nI have been working in the Data Science field, spearheading in machine learning, \ndeep learning, and computer vision. Typically this is my YouTube channel where I clarify \ndifferent themes on machine learning, deep learning, and AI with numerous real-world issue scenarios. \nMy primary point is to create everybody commonplace of ML and AI. Please subscribe and support the channel. \nAs I come up with new innovations. These recordings are free and I guarantee to create more curiously substance as we go ahead.\n😀Please donate if you want to support the channel through Buy me a coffee: https://www.buymeacoffee.com/dswithbappy\n\nP

In [12]:
len(response)

1

In [13]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [14]:
response = retriever.invoke("what is dswithbappy?")

In [15]:
response

[Document(metadata={'source': './data/dswithbappy_data.txt'}, page_content='dswithbappy focuses on providing content on Data Science, Artificial Intelligence, Machine Learning, \nDeep Learning, Computer Vision, Natural language processing, Python programming, etc. in English. \n"Real creativity won\'t make things more complex. Instead, I will simplify them." \nI have been working in the Data Science field, spearheading in machine learning, \ndeep learning, and computer vision. Typically this is my YouTube channel where I clarify \ndifferent themes on machine learning, deep learning, and AI with numerous real-world issue scenarios. \nMy primary point is to create everybody commonplace of ML and AI. Please subscribe and support the channel. \nAs I come up with new innovations. These recordings are free and I guarantee to create more curiously substance as we go ahead.\n😀Please donate if you want to support the channel through Buy me a coffee: https://www.buymeacoffee.com/dswithbappy\n\nP

## Simple use with LCEL

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()


In [18]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [19]:
response = chain.invoke("what is dswithbappy?")

In [20]:
response

'dswithbappy is a YouTube channel that focuses on providing content on Data Science, Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, Natural language processing, Python programming, etc. The channel aims to simplify complex topics in these areas and create awareness about ML and AI. The creator of the channel also works in the Data Science field, specializing in machine learning, deep learning, and computer vision. The channel offers free videos and encourages donations for support.'

In [21]:
type(response)

str